In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

# TODO: Create a simple CNN

In [9]:
class CNN(nn.Module):
    def __init__(self,in_channels = 1,num_classes = 10):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels = 1,
            out_channels = 8,
            kernel_size = (3,3),
            stride = (1,1),
            padding = (1,1)
        ) # same convolution
        self.pool = nn.MaxPool2d(kernel_size =(2,2), stride = (2,2))
        self.conv2 = nn.Conv2d(
            in_channels = 8,
            out_channels = 16,
            kernel_size = (3,3),
            stride = (1,1),
            padding = (1,1)
        ) # same convolution
        self.fc1 = nn.Linear(16*7*7,num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)

        return x
 

In [10]:
model = CNN()
x = torch.randn(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [11]:
# Hyperparamters
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

In [15]:
train_dataset = datasets.MNIST(
    root='dataset/',
    train=True,
    transform=transforms.ToTensor(),
    download=False
)
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True
)

test_dataset = datasets.MNIST(
    root='dataset/',
    train=False,
    transform=transforms.ToTensor(),
    download=False
)
test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = True
)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
#Initialize network
model = CNN(in_channels=in_channels,num_classes=num_classes).to(device)

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [22]:
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        targets = targets.to(device)
        
        #forward
        score = model(data)
        loss = loss_fn(score,targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #optim step
        optimizer.step()


100%|██████████| 938/938 [00:24<00:00, 38.86it/s]


In [23]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.10
Accuracy on test set: 97.80
